In [1]:
import pandas as pd
import json
import numpyfrom Constants import Constants as np
from abc import ABC, abstractmethod
from collections.abc import Iterable

import Utils

In [26]:
#scripts to query the john hopkins gitub
#outputs a dataframe/csv with geoid, day, month, year, cases, and deaths for each day/couty within a time frame
def jh_data(url, start_date = '3/1/2020'):
    #reads the data on covid cases from the jh github on us covid cases/deaths 
    #https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
    df = pd.read_csv(url,
                    engine='python',
                    error_bad_lines= False)
    df = df.dropna()
    to_drop = [col for col in df.columns if '/' not in col] #keep only dates which are formatted mm/dd/yyyy for this as of writing
    df.loc[:,'GEOID'] = df.FIPS.apply(lambda x: Utils.stringify_fips(x)) #don't drip this
    df = df.drop(to_drop, axis=1)
#     df = df.set_index('GEOID')
    return df

def clean_covid_data(url, value_name, start_date, end_date):
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    covid_df = jh_data(url).set_index("GEOID")
    times = list(covid_df.columns)
    times = [t for t in times if end >= pd.to_datetime(t) >= start]
    covid_entrys = []
    for geoid, column in covid_df.iterrows():
        for time in times:
            ftime = pd.to_datetime(time)
            entry = {'GEOID': geoid, value_name: column[time], 'day': ftime.day, 'month': ftime.month, 'year': ftime.year}
            covid_entrys.append(entry)
    return pd.DataFrame(covid_entrys)

def jh_covid_df(start_date = '3/1/2020', end_date ='6/30/2020'):
    deaths = clean_covid_data(Constants.jh_deaths_url,'deaths',start_date, end_date)
    cases = clean_covid_data(Constants.jh_cases_url,'cases',start_date, end_date)
    covid_df = deaths.merge(cases,on=['GEOID','day','month','year'], how='inner')
    return covid_df

df = jh_covid_df()
df.T

,0,1,2,3,4,5,6,7,8,9,...,405518,405519,405520,405521,405522,405523,405524,405525,405526,405527
GEOID,01001,01001,01001,01001,01001,01001,01001,01001,01001,01001,...,56045,56045,56045,56045,56045,56045,56045,56045,56045,56045
day,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
deaths,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
month,3,3,3,3,3,3,3,3,3,3,...,6,6,6,6,6,6,6,6,6,6
year,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020,...,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020
cases,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,2


In [27]:
#save to covid_cases_and_deaths.csv
df.to_csv(Constants.covid_cases_output_file)